### 평가

In [2]:
import re
import os
import string
import collections
import pandas as pd
from collections import Counter
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [7]:
# results_file = 'results/result_20250122_q_QnA_car_sample_10.json'
results_file = 'results/result_20250131_q_QnA_car_sample_30_context_labeling_markdown.json'

results = pd.read_json(results_file)
methods = list(results['llm response'][0].keys())

#### Hit Rate

##### page version

In [ ]:
for method in methods:
    predictions = []
    references = []
    print(f"method: {method}")
    for i in range(len(results["llm response"].keys())):
        ref_pg = []
        ref_pg.append(int(results['ground truth answer pages'][i].strip('[]')))
        references.append(ref_pg)
        predictions.append(results['llm response'][i][method]['pages'])

    # Flatten the references and predictions
    y_true = []
    y_pred = []        
    # Convert references and predictions to binary 1D arrays for evaluation
    for ref, pred in zip(references, predictions):
        # Flatten each reference into binary labels
        y_true.append([1 if r in pred else 0 for r in ref])
        y_pred.append([1] * len(ref))  # Always assume predictions are 'positive'

    # Calculate Precision
    total_correct = sum([1 if 1 in true else 0 for true in y_true])
    precision = total_correct / len(references)

    print(f"Hit Rate: {precision:.3f}")


##### context version

In [10]:
for method in methods:
    predictions = []
    references = []
    print(f"[Method] {method}")
    
    # 모든 데이터 먼저 수집
    for i in range(len(results["llm response"].keys())):
        references.append(results['ground truth answer context'][i])
        predictions.append(results['llm response'][i][method]['page contents'])
    
    # 평가 진행
    y_true = []
    y_pred = []
    for ref, pred_contents in zip(references, predictions):
        # 각 reference에 대해 예측된 페이지들과 비교
        y_true.append([1 if ref in content else 0 for content in pred_contents])
        y_pred.append([1] * len(pred_contents))
    
    # Hit Rate 계산
    total_correct = sum([1 if 1 in true else 0 for true in y_true])
    precision = total_correct / len(references)

    print(f"Hit Rate: {precision:.3f}")
    print('-'*100)

[Method] dense_sts_chunking_500_100_rerank_3
Hit Rate: 0.207
----------------------------------------------------------------------------------------------------
[Method] dense_sts_chunking_500_100_no_reranker
Hit Rate: 0.207
----------------------------------------------------------------------------------------------------
[Method] sparse_bm25_chunking_500_100_rerank_3
Hit Rate: 0.000
----------------------------------------------------------------------------------------------------
[Method] sparse_bm25_chunking_500_100_no_reranker
Hit Rate: 0.000
----------------------------------------------------------------------------------------------------
[Method] dense_sts_full_page_rerank_3
Hit Rate: 0.241
----------------------------------------------------------------------------------------------------
[Method] dense_sts_full_page_no_reranker
Hit Rate: 0.276
----------------------------------------------------------------------------------------------------
[Method] sparse_bm25_full_pag

#### Micro F1

In [11]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()


def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))


def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

In [12]:
def calculate_micro_f1(references, predictions):
    total_tp = 0  # 전체 True Positives
    total_fp = 0  # 전체 False Positives
    total_fn = 0  # 전체 False Negatives

    for ref, pred in zip(references, predictions):
        # 각 예제의 단어 집합 추출
        ref_set = set(ref.split())
        pred_set = set(pred.split())

        # TP, FP, FN 계산
        tp = len(ref_set & pred_set)  # 교집합: 정확히 맞춘 단어 수
        fp = len(pred_set - ref_set)  # 예측에는 있지만 참조에는 없는 단어 수
        fn = len(ref_set - pred_set)  # 참조에는 있지만 예측에는 없는 단어 수

        # 누적
        total_tp += tp
        total_fp += fp
        total_fn += fn

    # Precision, Recall 계산
    precision = total_tp / (total_tp + total_fp) if total_tp + total_fp > 0 else 0
    recall = total_tp / (total_tp + total_fn) if total_tp + total_fn > 0 else 0

    # Micro F1 Score 계산
    micro_f1 = (2 * precision * recall) / (precision + recall) if precision + recall > 0 else 0

    return micro_f1

In [13]:
# make lists of prediction and reference
def make_lists(results, method):
    predictions = []
    references = []
    for i in range(len(list(results['llm response'].keys()))):
        predictions.append(results['llm response'][i][method]['answer'])
        references.append(results['ground truth answer'][i])
    return predictions, references

In [14]:
# evaluate micro f1 score
methods = list(results['llm response'][0].keys())
for method in methods:
    print(f"[Method] {method}")
    predictions, references = make_lists(results, method)
    print(f"micro f1 score: {round(calculate_micro_f1(references, predictions), 3)}")
    print('-'*100)


[Method] dense_sts_chunking_500_100_rerank_3
micro f1 score: 0.226
----------------------------------------------------------------------------------------------------
[Method] dense_sts_chunking_500_100_no_reranker
micro f1 score: 0.23
----------------------------------------------------------------------------------------------------
[Method] sparse_bm25_chunking_500_100_rerank_3
micro f1 score: 0.176
----------------------------------------------------------------------------------------------------
[Method] sparse_bm25_chunking_500_100_no_reranker
micro f1 score: 0.165
----------------------------------------------------------------------------------------------------
[Method] dense_sts_full_page_rerank_3
micro f1 score: 0.156
----------------------------------------------------------------------------------------------------
[Method] dense_sts_full_page_no_reranker
micro f1 score: 0.216
-----------------------------------------------------------------------------------------------

#### GPT-4 Judge

In [23]:
# evaluate generation using LLM as a judge
import os
from modules.config_loader import load_config
from typing_extensions import Annotated, TypedDict
from langchain_openai import ChatOpenAI

config = load_config('config/config.yaml')

os.environ["LANGCHAIN_TRACING_V2"] = config['langchain']['tracing_v2']
os.environ["LANGCHAIN_API_KEY"] = config['langchain']['api_key']
os.environ["OPENAI_API_KEY"] = config['openai']['api_key']

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
# Grade output schema
class CorrectnessGrade(TypedDict):
    # Note that the order in the fields are 
    # defined is the order in which the model will generate them.
    # It is useful to put explanations before responses because it forces the model to think through
    # its final response before generating it:
    explanation: Annotated[str, ..., "왜 이 점수를 주었는지 설명해주세요."]
    correct: Annotated[bool, ..., "답변이 옳다면 True, 틀리다면 False"]

# Grade prompt
# add explanation prompt
correctness_instructions = f"""당신은 문제를 채점하는 교사입니다. 

질문, 정답, 그리고 학생 답변이 주어질 것입니다.

채점 기준은 다음과 같습니다:
1. 학생 답변을 정답과 비교하여 사실적 정확성만을 평가합니다.
2. 학생 답변에 상충되는 진술이 포함되지 않았는지 확인하십시오.
3. 학생 답변이 정답보다 더 많은 정보를 포함하더라도, 정답과 비교했을 때 사실적으로 정확하다면 정답으로 평가합니다.

정답 여부:
정답 여부가 True라면, 학생 답변이 모든 채점 기준을 충족했음을 의미합니다.
정답 여부가 False라면, 학생 답변이 모든 채점 기준을 충족하지 않았음을 의미합니다.

답변을 단계별로 설명하여 추론과 결론이 올바른지 확인하십시오.

처음부터 정답을 단순히 말하지 마십시오.
"""

# Grader LLM
grader_llm = ChatOpenAI(model=config['openai']['gpt_model'], temperature=0).with_structured_output(CorrectnessGrade, method="json_schema", strict=True)

def correctness(inputs: str, outputs: str, reference_outputs: str) -> bool:
    """An evaluator for RAG answer accuracy"""
    answers = f"""질문: {inputs}
정답: {reference_outputs}
학생 답변: {outputs}
"""

    # Run evaluator
    grade = grader_llm.invoke([{"role": "system", "content": correctness_instructions}, {"role": "user", "content": answers}])
    return grade["correct"], grade["explanation"]

In [51]:
methods = list(results['llm response'][0].keys())
eval_results = []
for method in methods:
    for i in range(len(results['number'])):
        judge, explanation = correctness(results['question'][i], results['llm response'][i][method], results['ground truth answer'][i])
        
        eval = {
            'question': results['question'][i],
            'ground truth answer': results['ground truth answer'][i],
            'student answer': results['llm response'][i][method]['answer'],
            'method': method,
            'judge': judge,
            'explanation': explanation
        }
        eval_results.append(eval)
        
        print(f"방법: {method}")
        print(f"질문: {results['question'][i]}")
        print(f"정답: {results['ground truth answer'][i]}")
        print(f"학생 답변: {results['llm response'][i][method]['answer']}")
        print(f"GPT-4o 평가: {judge} - {explanation}")
        print('-'*100)


방법: dense_sts_chunking_500_100_rerank_3
질문: 펫케어 특별약관에 가입했는데 저희 반려견 등록번호를 알려드려야 하나요?
정답: 네. 보험계약자 또는 피보험자는 이 특별약관의 계약체결 시 가입하고자 하는 반려 동물의 국가동물보호정보시스템 상 동물등록번호를 보험회사에 알려주어야 합니다.
학생 답변: 네, 펫케어 특별약관에 가입할 때 반려동물의 국가동물보호정보시스템 상 동물등록번호를 보험회사에 알려주어야 합니다.
GPT-4o 평가: True - 학생의 답변은 정답과 사실적으로 일치합니다. 학생은 반려동물의 국가동물보호정보시스템 상 동물등록번호를 보험회사에 알려주어야 한다고 정확히 언급하였습니다. 추가적인 페이지 정보와 내용은 답변의 정확성에 영향을 미치지 않습니다.
----------------------------------------------------------------------------------------------------
방법: dense_sts_chunking_500_100_rerank_3
질문: 자동차보험을 철회할 수 있나요?
정답: 일반금융소비자는 보험증권을 받은 날부터 15일과 청약을 한 날부터 30 일 중 먼저 도래하는 기간 내에 보험계약의 청약을 철회할 수 있습니다.
학생 답변: 자동차보험을 철회할 수 있는 경우는 특정 조건을 충족해야 합니다. 예를 들어, 장기간 자동차를 운행하지 않을 계획이라면, 관할 관청에 사유를 신고하고 승인을 받아 등록증과 번호판을 반납하면 보험 가입이 면제될 수 있습니다. 그러나 일반적인 경우에는 보험을 철회하기 어렵습니다. 추가적인 조건이나 절차에 대한 정보가 필요하시면 보험사에 직접 문의하시는 것이 좋습니다.
GPT-4o 평가: False - 학생의 답변은 자동차보험 철회에 대한 일반적인 규정이 아닌, 특정 조건 하에서 보험 가입이 면제될 수 있는 경우에 대한 설명입니다. 정답은 일반금융소비자가 보험증권을 받은 날부터 15일과 청약을 한 날부터 30일 중 먼저 도래하는 기간

In [53]:
# make excel file
eval_results_df = pd.DataFrame(eval_results)
eval_results_df.to_excel(f'evaluation_results/250131_gpt4-judge_correctness.xlsx', index=True, header=True)


In [54]:
methods = list(eval_results_df['method'].unique())
# compute True(boolean) proportion
for method in methods:
    print(f"[Method] {method}")
    true_proportion = eval_results_df[eval_results_df['method'] == method]['judge'].value_counts(normalize=True)[0]
    print(f"True proportion: {true_proportion:.3f}")
    print('-'*100)

[Method] dense_sts_chunking_500_100_rerank_3
True proportion: 0.690
----------------------------------------------------------------------------------------------------
[Method] dense_sts_chunking_500_100_no_reranker
True proportion: 0.724
----------------------------------------------------------------------------------------------------
[Method] sparse_bm25_chunking_500_100_rerank_3
True proportion: 0.586
----------------------------------------------------------------------------------------------------
[Method] sparse_bm25_chunking_500_100_no_reranker
True proportion: 0.655
----------------------------------------------------------------------------------------------------
[Method] dense_sts_full_page_rerank_3
True proportion: 0.517
----------------------------------------------------------------------------------------------------
[Method] dense_sts_full_page_no_reranker
True proportion: 0.724
----------------------------------------------------------------------------------------

/tmp/ipykernel_11684/331474300.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  true_proportion = eval_results_df[eval_results_df['method'] == method]['judge'].value_counts(normalize=True)[0]


In [61]:
# Grade output schema
class RelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[bool, ..., "Provide the score on whether the answer addresses the question"]

# Grade prompt
relevance_instructions="""당신은 문제를 채점하는 교사입니다.

질문과 학생 답변이 주어질 것입니다.

채점 기준은 다음과 같습니다:
1. 학생 답변이 질문에 대해 간결하고 적절한지 확인하세요.
2. 학생 답변이 질문에 대한 답을 제공하는 데 도움이 되는지 확인하세요.

관련성:
관련성이 True일 경우, 학생의 답변이 위의 기준을 모두 충족했음을 의미합니다.
관련성이 False일 경우, 학생의 답변이 위의 기준을 모두 충족하지 못했음을 의미합니다.

답변을 단계별로 설명하여 추론과 결론이 올바른지 확인하시오.

처음부터 정답을 단순히 말하지 마시오.

답변은 한국어로 제시하시오.
"""

# Grader LLM
relevance_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(RelevanceGrade, method="json_schema", strict=True)

# Evaluator
def relevance(inputs: str, outputs: str) -> bool:
    """A simple evaluator for RAG answer helpfulness."""
    answer = f"""      질문: {inputs}
학생 답변: {outputs}"""
    grade = relevance_llm.invoke([{"role": "system", "content": relevance_instructions}, {"role": "user", "content": answer}])
    return grade["relevant"], grade["explanation"]

In [62]:
methods = list(results['llm response'][0].keys())
eval_results = []
for method in methods:
    for i in range(len(results['number'])):
        judge, explanation = relevance(results['question'][i], results['llm response'][i][method])
        
        eval = {
            'question': results['question'][i],
            'ground truth answer': results['ground truth answer'][i],
            'student answer': results['llm response'][i][method]['answer'],
            'method': method,
            'judge': judge,
            'explanation': explanation
        }
        eval_results.append(eval)
        
        print(f"방법: {method}")
        print(f"질문: {results['question'][i]}")
        print(f"학생 답변: {results['llm response'][i][method]['answer']}")
        print(f"GPT-4o 평가: {judge} - {explanation}")
        print('-'*100)

# make excel file
eval_results_df = pd.DataFrame(eval_results)
eval_results_df.to_excel(f'evaluation_results/250131_gpt4-judge_relevance.xlsx', index=True, header=True)

방법: dense_sts_chunking_500_100_rerank_3
질문: 펫케어 특별약관에 가입했는데 저희 반려견 등록번호를 알려드려야 하나요?
학생 답변: 네, 펫케어 특별약관에 가입할 때 반려동물의 국가동물보호정보시스템 상 동물등록번호를 보험회사에 알려주어야 합니다.
GPT-4o 평가: True - 학생의 답변은 질문에 대한 명확하고 간결한 답변을 제공하고 있습니다. 질문은 펫케어 특별약관에 가입할 때 반려견의 등록번호를 알려야 하는지에 대한 것이었고, 학생은 이에 대해 '네'라고 답변하며, 국가동물보호정보시스템 상의 동물등록번호를 보험회사에 알려주어야 한다고 설명하고 있습니다. 이는 질문에 대한 적절한 답변입니다.
----------------------------------------------------------------------------------------------------
방법: dense_sts_chunking_500_100_rerank_3
질문: 자동차보험을 철회할 수 있나요?
학생 답변: 자동차보험을 철회할 수 있는 경우는 특정 조건을 충족해야 합니다. 예를 들어, 장기간 자동차를 운행하지 않을 계획이라면, 관할 관청에 사유를 신고하고 승인을 받아 등록증과 번호판을 반납하면 보험 가입이 면제될 수 있습니다. 그러나 일반적인 경우에는 보험을 철회하기 어렵습니다. 추가적인 조건이나 절차에 대한 정보가 필요하시면 보험사에 직접 문의하시는 것이 좋습니다.
GPT-4o 평가: True - 학생의 답변은 자동차보험을 철회할 수 있는지에 대한 질문에 대해 적절한 정보를 제공하고 있습니다. 답변은 특정 조건 하에서 보험 철회가 가능하다는 점을 설명하고 있으며, 일반적인 경우에는 철회가 어렵다는 점도 명확히 하고 있습니다. 또한, 추가적인 정보가 필요할 경우 보험사에 문의하라는 조언도 포함되어 있어 질문에 대한 답변으로 적절합니다.
-------------------------------------------------------------

In [64]:
methods = list(eval_results_df['method'].unique())
# compute True(boolean) proportion
for method in methods:
    print(f"[Method] {method}")
    true_proportion = eval_results_df[eval_results_df['method'] == method]['judge'].value_counts(normalize=True)[0]
    print(f"True proportion: {true_proportion:.3f}")
    print('-'*100)

[Method] dense_sts_chunking_500_100_rerank_3
True proportion: 0.862
----------------------------------------------------------------------------------------------------
[Method] dense_sts_chunking_500_100_no_reranker
True proportion: 0.793
----------------------------------------------------------------------------------------------------
[Method] sparse_bm25_chunking_500_100_rerank_3
True proportion: 0.517
----------------------------------------------------------------------------------------------------
[Method] sparse_bm25_chunking_500_100_no_reranker
True proportion: 0.552
----------------------------------------------------------------------------------------------------
[Method] dense_sts_full_page_rerank_3
True proportion: 0.690
----------------------------------------------------------------------------------------------------
[Method] dense_sts_full_page_no_reranker
True proportion: 0.759
----------------------------------------------------------------------------------------

/tmp/ipykernel_11684/331474300.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  true_proportion = eval_results_df[eval_results_df['method'] == method]['judge'].value_counts(normalize=True)[0]


In [68]:
# Grade output schema
class GroundedGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    grounded: Annotated[bool, ..., "Provide the score on if the answer hallucinates from the documents"]

# Grade prompt
grounded_instructions = """당신은 문제를 채점하는 교사입니다. 

사실과 학생 답변이 주어질 것입니다.

채점 기준은 다음과 같습니다:
1. 학생 답변이 주어진 사실에 근거하고 있는지 확인하세요.
2. 학생 답변이 주어진 사실에 포함되지 않은 "허위 정보"를 포함하지 않았는지 확인하세요.

근거 여부:
근거 여부가 True라면, 학생 답변이 모든 채점 기준을 충족했음을 의미합니다.
근거 여부가 False라면, 학생 답변이 모든 채점 기준을 충족하지 않았음을 의미합니다.

답변을 단계별로 설명하여 추론과 결론이 올바른지 확인하시오.

처음부터 정답을 단순히 말하지 마시오.

답변은 한국어로 제시하시오.
"""

# Grader LLM 
grounded_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(GroundedGrade, method="json_schema", strict=True)

# Evaluator
def groundedness(inputs: str, contents: list) -> bool:
    """A simple evaluator for RAG answer groundedness."""
    doc_string = "".join(contents)
    answer = f"""      사실: {doc_string}
학생 답변: {inputs}"""
    grade = grounded_llm.invoke([{"role": "system", "content": grounded_instructions}, {"role": "user", "content": answer}])
    return grade["grounded"], grade["explanation"]

In [70]:
methods = list(results['llm response'][0].keys())
eval_results = []
for method in methods:
    for i in range(len(results['number'])):
        judge, explanation = groundedness(results['question'][i], results['llm response'][i][method]['page contents'])
        
        eval = {
            'question': results['question'][i],
            'student answer': results['llm response'][i][method]['answer'],
            'page contents': results['llm response'][i][method]['page contents'],
            'method': method,
            'judge': judge,
            'explanation': explanation
        }
        eval_results.append(eval)
        
        print(f"방법: {method}")
        print(f"질문: {results['question'][i]}")
        print(f"학생 답변: {results['llm response'][i][method]['answer']}")
        print(f"GPT-4o 평가: {judge} - {explanation}")
        print('-'*100)

# make excel file
eval_results_df = pd.DataFrame(eval_results)
eval_results_df.to_excel(f'evaluation_results/250131_gpt4-judge_groundedness.xlsx', index=True, header=True)

방법: dense_sts_chunking_500_100_rerank_3
질문: 펫케어 특별약관에 가입했는데 저희 반려견 등록번호를 알려드려야 하나요?
학생 답변: 네, 펫케어 특별약관에 가입할 때 반려동물의 국가동물보호정보시스템 상 동물등록번호를 보험회사에 알려주어야 합니다.
GPT-4o 평가: True - 학생의 질문은 펫케어 특별약관에 가입할 때 반려견 등록번호를 알려야 하는지에 대한 것입니다. 사실에 따르면, 제3조(계약 전 알릴의무)에서 보험계약자 또는 피보험자는 계약체결 시 반려동물의 국가동물보호정보시스템 상 동물등록번호를 보험회사에 알려주어야 한다고 명시되어 있습니다. 따라서 학생의 질문은 사실에 근거하고 있으며, 주어진 정보에 부합합니다. 학생의 답변은 허위 정보를 포함하지 않았습니다.
----------------------------------------------------------------------------------------------------
방법: dense_sts_chunking_500_100_rerank_3
질문: 자동차보험을 철회할 수 있나요?
학생 답변: 자동차보험을 철회할 수 있는 경우는 특정 조건을 충족해야 합니다. 예를 들어, 장기간 자동차를 운행하지 않을 계획이라면, 관할 관청에 사유를 신고하고 승인을 받아 등록증과 번호판을 반납하면 보험 가입이 면제될 수 있습니다. 그러나 일반적인 경우에는 보험을 철회하기 어렵습니다. 추가적인 조건이나 절차에 대한 정보가 필요하시면 보험사에 직접 문의하시는 것이 좋습니다.
GPT-4o 평가: False - 학생의 질문은 자동차보험을 철회할 수 있는지에 대한 것입니다. 주어진 사실에서는 자동차보험의 의무가입 면제 사유와 관련된 내용이 주로 설명되어 있으며, 보험을 철회할 수 있는지에 대한 직접적인 정보는 포함되어 있지 않습니다. 따라서 학생의 질문은 주어진 사실에 근거하지 않았습니다.
----------------------------------------------------

In [72]:
methods = list(eval_results_df['method'].unique())
# compute True(boolean) proportion
for method in methods:
    print(f"[Method] {method}")
    true_proportion = eval_results_df[eval_results_df['method'] == method]['judge'].value_counts(normalize=True)[0]
    print(f"True proportion: {true_proportion:.3f}")
    print('-'*100)

[Method] dense_sts_chunking_500_100_rerank_3
True proportion: 0.655
----------------------------------------------------------------------------------------------------
[Method] dense_sts_chunking_500_100_no_reranker
True proportion: 0.759
----------------------------------------------------------------------------------------------------
[Method] sparse_bm25_chunking_500_100_rerank_3
True proportion: 0.586
----------------------------------------------------------------------------------------------------
[Method] sparse_bm25_chunking_500_100_no_reranker
True proportion: 0.552
----------------------------------------------------------------------------------------------------
[Method] dense_sts_full_page_rerank_3
True proportion: 0.586
----------------------------------------------------------------------------------------------------
[Method] dense_sts_full_page_no_reranker
True proportion: 0.621
----------------------------------------------------------------------------------------

/tmp/ipykernel_11684/331474300.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  true_proportion = eval_results_df[eval_results_df['method'] == method]['judge'].value_counts(normalize=True)[0]


In [77]:
# Grade output schema
class RetrievalRelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[bool, ..., "True if the retrieved documents are relevant to the question, False otherwise"]

# Grade prompt
retrieval_relevance_instructions = """당신은 문제를 채점하는 교사입니다.

질문과 학생이 제공한 사실이 주어질 것입니다.

채점 기준은 다음과 같습니다.
1. 당신의 목표는 질문과 완전히 무관한 사실을 식별하는 것입니다.
2. 만약 사실이 질문과 관련된 키워드나 의미를 조금이라도 포함하고 있다면, 해당 사실을 관련성이 있다고 평가하세요.
3. 사실과 무관한 정보가 일부 포함되어 있더라도, 위의 2. 기준을 충복한다면 관련성이 있다고 판단하세요.

관련성:
관련성이 True라면, 사실이 질문과 관련된 키워드나 의미를 조금이라도 포함하고 있음을 의미합니다.
관련성이 False라면, 사실이 질문과 완전히 무관함을 의미합니다.

답변을 단계별로 설명하여 추론과 결론이 올바른지 확인하시오.

처음부터 정답을 단순히 말하지 마시오.

답변은 한국어로 제시하시오."""

# Grader LLM
retrieval_relevance_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(RetrievalRelevanceGrade, method="json_schema", strict=True)

def retrieval_relevance(inputs: str, contents: list) -> bool:
    """An evaluator for document relevance"""
    doc_string = "".join(contents)
    answer = f"""      사실: {doc_string}
질문: {inputs}"""
    grade = retrieval_relevance_llm.invoke([{"role": "system", "content": retrieval_relevance_instructions}, {"role": "user", "content": answer}])
    return grade["relevant"], grade["explanation"]

In [81]:
methods = list(results['llm response'][0].keys())
eval_results = []
for method in methods:
    for i in range(len(results['number'])):
        judge, explanation = retrieval_relevance(results['question'][i], results['llm response'][i][method]['page contents'])
        
        eval = {
            'question': results['question'][i],
            'page contents': results['llm response'][i][method]['page contents'],
            'method': method,
            'judge': judge,
            'explanation': explanation
        }
        eval_results.append(eval)
        
        facts = "".join(results['llm response'][i][method]['page contents'])
        print(f"방법: {method}")
        print(f"질문: {results['question'][i]}")
        print(f"사실: {facts}")
        print(f"GPT-4o 평가: {judge} - {explanation}")
        print('-'*100)

# make excel file
eval_results_df = pd.DataFrame(eval_results)
eval_results_df.to_excel(f'evaluation_results/250131_gpt4-judge_retrieval-relevance.xlsx', index=True, header=True)

방법: dense_sts_chunking_500_100_rerank_3
질문: 펫케어 특별약관에 가입했는데 저희 반려견 등록번호를 알려드려야 하나요?
사실: 용어풀이

  


① '폐사'라 함은 보통약관 제21조(보상하는 손해)에서 정하는 사고의 직접적인 결  
과로 수의사법 제12조(진단서 등)에 따라 수의사가 발급한 폐사 진단서에 의해  
반려동물의 폐사가 확인된 경우를 말합니다.  
② '부상'이라 함은 보통약관 제21조(보상하는 손해)에서 정하는 사고의 직접적인  
결과로 반려동물이 상해를 입어 수의사법 제12조(진단서 등)에 따라 수의사가  
발급한 진단서에 의해 부상이 확인된 경우를 말합니다.

# 제3조(계약 전 알릴의무)

  


보험계약자 또는 피보험자는 이 특별약관의 계약체결 시 가입하고자 하는 반려  
동물의 국가동물보호정보시스템 상 동물등록번호를 보험회사에 알려주어야 합  
니다.

제4조(보상하지 않는 손해)

  


보험회사는 보통약관 제23조(보상하지 않는 손해)에 해당하는 사항 이외에 다음  
과 같은 손해에 대하여도 보험금을 지급하지 않습니다.# 3\. 동물등록증

  


4\. 가족관계증명서 (등록된 반려동물이 기명피보험자의 가족 소유로 등록된  
경우에 한함)

  


5\. 그 밖에 보험회사가 꼭 필요하다고 인정하는 서류 또는 증거

  


# 제7조(준용규정)

  


이 특별약관에서 정하지 아니한 사항은 보통약관에 따릅니다.

150 특별약관  


형사합의금 지원 특별약관Ⅱ

  


# 제6편 사고처리시 필요 비용지원

제1조(적용대상)

  


이 특별약관은 보통약관 「대인배상Ⅱ」를 가입하고 「형사합의금 지원 특별약관  
Ⅱ」에 가입한 경우에 적용됩니다.

  


제2조(보상내용)

  


보험회사는 피보험자가 피보험자동차를 소유, 사용, 관리하는 동안에 생긴 피보  
험자동차의 사고로 인하여 형사상 책임을 지거나, 「형법」 제258조 제1항 또는  
제2항, 「교통사고처리특례법」 제4조 제1항 제

In [82]:
methods = list(eval_results_df['method'].unique())
# compute True(boolean) proportion
for method in methods:
    print(f"[Method] {method}")
    true_proportion = eval_results_df[eval_results_df['method'] == method]['judge'].value_counts(normalize=True)[0]
    print(f"True proportion: {true_proportion:.3f}")
    print('-'*100)

[Method] dense_sts_chunking_500_100_rerank_3
True proportion: 0.862
----------------------------------------------------------------------------------------------------
[Method] dense_sts_chunking_500_100_no_reranker
True proportion: 0.931
----------------------------------------------------------------------------------------------------
[Method] sparse_bm25_chunking_500_100_rerank_3
True proportion: 0.552
----------------------------------------------------------------------------------------------------
[Method] sparse_bm25_chunking_500_100_no_reranker
True proportion: 0.586
----------------------------------------------------------------------------------------------------
[Method] dense_sts_full_page_rerank_3
True proportion: 0.828
----------------------------------------------------------------------------------------------------
[Method] dense_sts_full_page_no_reranker
True proportion: 0.793
----------------------------------------------------------------------------------------

/tmp/ipykernel_11684/331474300.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  true_proportion = eval_results_df[eval_results_df['method'] == method]['judge'].value_counts(normalize=True)[0]


#### ROUGE-L

In [15]:
# evaluation ROUGE-L score
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
methods = list(results['llm response'][0].keys())
for method in methods:
    print(f"[Method] {method}")
    predictions, references = make_lists(results, method)
    rouge_l_scores = []
    for ref, pred in zip(references, predictions):
        score = scorer.score(pred, ref)
        rouge_l_scores.append(score['rougeL'].fmeasure)
    
    average_rouge_l_score = sum(rouge_l_scores) / len(rouge_l_scores)
    print(f"average rouge-l score: {average_rouge_l_score:.3f}")
    print('-'*100)



[Method] dense_sts_chunking_500_100_rerank_3
average rouge-l score: 0.190
----------------------------------------------------------------------------------------------------
[Method] dense_sts_chunking_500_100_no_reranker
average rouge-l score: 0.184
----------------------------------------------------------------------------------------------------
[Method] sparse_bm25_chunking_500_100_rerank_3
average rouge-l score: 0.161
----------------------------------------------------------------------------------------------------
[Method] sparse_bm25_chunking_500_100_no_reranker
average rouge-l score: 0.149
----------------------------------------------------------------------------------------------------
[Method] dense_sts_full_page_rerank_3
average rouge-l score: 0.138
----------------------------------------------------------------------------------------------------
[Method] dense_sts_full_page_no_reranker
average rouge-l score: 0.195
----------------------------------------------------

#### BLEU

In [16]:
# evaluation BLEU score

methods = list(results['llm response'][0].keys())
smooth_fn = SmoothingFunction().method1

for method in methods:
    print(f"[Method] {method}")
    predictions, references = make_lists(results, method)
    bleu_scores = []
    for ref, pred in zip(references, predictions):
        score = sentence_bleu([ref], pred, smoothing_function=smooth_fn)
        bleu_scores.append(score)
    average_bleu_score = sum(bleu_scores) / len(bleu_scores)
    print(f"average bleu score: {average_bleu_score:.3f}")
    print('-'*100)


[Method] dense_sts_chunking_500_100_rerank_3
average bleu score: 0.242
----------------------------------------------------------------------------------------------------
[Method] dense_sts_chunking_500_100_no_reranker
average bleu score: 0.254
----------------------------------------------------------------------------------------------------
[Method] sparse_bm25_chunking_500_100_rerank_3
average bleu score: 0.182
----------------------------------------------------------------------------------------------------
[Method] sparse_bm25_chunking_500_100_no_reranker
average bleu score: 0.170
----------------------------------------------------------------------------------------------------
[Method] dense_sts_full_page_rerank_3
average bleu score: 0.186
----------------------------------------------------------------------------------------------------
[Method] dense_sts_full_page_no_reranker
average bleu score: 0.227
----------------------------------------------------------------------

#### Fail Rate


In [17]:
fail_sentence = "죄송합니다. 해당 질문에 대한 답변을 찾을 수 없습니다."

methods = list(results['llm response'][0].keys())
for method in methods:
    print(f"[Method] {method}")
    predictions, _ = make_lists(results, method)
    fail_rate = sum(1 for pred in predictions if fail_sentence in pred) / len(references)
    print(f"fail rate: {fail_rate:.3f}")
    print('-'*100)

[Method] dense_sts_chunking_500_100_rerank_3
fail rate: 0.138
----------------------------------------------------------------------------------------------------
[Method] dense_sts_chunking_500_100_no_reranker
fail rate: 0.207
----------------------------------------------------------------------------------------------------
[Method] sparse_bm25_chunking_500_100_rerank_3
fail rate: 0.517
----------------------------------------------------------------------------------------------------
[Method] sparse_bm25_chunking_500_100_no_reranker
fail rate: 0.552
----------------------------------------------------------------------------------------------------
[Method] dense_sts_full_page_rerank_3
fail rate: 0.345
----------------------------------------------------------------------------------------------------
[Method] dense_sts_full_page_no_reranker
fail rate: 0.241
----------------------------------------------------------------------------------------------------
[Method] sparse_bm25_fu

#### NDCG

In [18]:
# evaluation NDCG score
full_pages = [method for method in methods if 'full_page' in method]

In [19]:
import numpy as np

def dcg_at_k(relevance_scores, k):
    relevance_scores = np.array(relevance_scores)[:k]
    return np.sum(relevance_scores / np.log2(np.arange(2, len(relevance_scores) + 2)))

def ndcg_at_k(predicted, ground_truth, k):
    
    # Relevance 점수 계산 (ground truth가 Top-K에 포함된 위치를 확인)
    relevance_scores = [1 if pred == ground_truth[0] else 0 for pred in predicted]
    
    # DCG 계산
    dcg = dcg_at_k(relevance_scores, k)
    
    # IDCG 계산 (ground truth가 첫 번째에 있을 때)
    ideal_relevance_scores = [1] + [0] * (k - 1)
    idcg = dcg_at_k(ideal_relevance_scores, k)
    
    # NDCG 계산
    return dcg / idcg if idcg > 0 else 0

In [20]:
import numpy as np

def dcg_at_k(relevance_scores, k):
    """
    Calculate DCG@k
    :param relevance_scores: List of relevance scores for the predictions
    :param k: Rank position up to which DCG is calculated
    :return: DCG@k value
    """
    relevance_scores = np.array(relevance_scores)[:k]
    return np.sum(relevance_scores / np.log2(np.arange(2, len(relevance_scores) + 2)))

def ndcg_at_k(predicted, ground_truth, k):
    """
    Calculate NDCG@k
    :param predicted: List of predicted outputs
    :param ground_truth: List of ground truth values (can have length >= 1)
    :param k: Rank position up to which NDCG is calculated
    :return: NDCG@k value
    """
    # Relevance 점수 계산 (ground truth 중 하나라도 일치하면 relevance 점수를 1로 설정)
    relevance_scores = [1 if pred in ground_truth else 0 for pred in predicted]

    # DCG 계산
    dcg = dcg_at_k(relevance_scores, k)

    # IDCG 계산 (ground truth가 이상적인 위치에 있다고 가정)
    ideal_relevance_scores = [1] * min(len(ground_truth), k) + [0] * (k - len(ground_truth))
    idcg = dcg_at_k(ideal_relevance_scores, k)

    # NDCG 계산
    return dcg / idcg if idcg > 0 else 0

In [22]:
k = 3

for method in methods:
    print(f"[Method] {method}")
    predictions = []
    references = []
    ndcg_scores = []
    
    # 데이터 수집
    for i in range(len(results['llm response'].keys())):
        ref_context = results['ground truth answer context'][i]
        references.append(ref_context)
        
        # 예측된 페이지 컨텐츠 가져오기
        predictions.append(results['llm response'][i][method]['page contents'])
    
    # NDCG 계산
    for ref, pred_contents in zip(references, predictions):
        # 각 페이지 컨텐츠에 대해 reference가 포함되어 있는지 확인 (relevance score 계산)
        relevance_scores = [1 if ref in content else 0 for content in pred_contents]
        score = ndcg_at_k(relevance_scores, [1], k)  # ideal case는 첫 번째가 정답
        ndcg_scores.append(score)
        
    average_ndcg_score = sum(ndcg_scores) / len(ndcg_scores)
    print(f"average ndcg@{k} score: {average_ndcg_score:.3f}")
    print('-'*100)

[Method] dense_sts_chunking_500_100_rerank_3
average ndcg@3 score: 0.177
----------------------------------------------------------------------------------------------------
[Method] dense_sts_chunking_500_100_no_reranker
average ndcg@3 score: 0.199
----------------------------------------------------------------------------------------------------
[Method] sparse_bm25_chunking_500_100_rerank_3
average ndcg@3 score: 0.000
----------------------------------------------------------------------------------------------------
[Method] sparse_bm25_chunking_500_100_no_reranker
average ndcg@3 score: 0.000
----------------------------------------------------------------------------------------------------
[Method] dense_sts_full_page_rerank_3
average ndcg@3 score: 0.203
----------------------------------------------------------------------------------------------------
[Method] dense_sts_full_page_no_reranker
average ndcg@3 score: 0.203
----------------------------------------------------------